In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib

# Set backend to TkAgg if not running in Jupyter or similar
matplotlib.use('TkAgg')

# Load your data
data = pd.read_csv('Retail_Food_Stores_20241010.csv')

# List of target zip codes
target_zipcodes = ['10036', '10001', '10012', '11101', '11206', '11211', '11249']

# Filter for stores in the target zip codes
filtered_data = data[data['Zip Code'].astype(str).isin(target_zipcodes)]

In [2]:
# Count stores in each zip code
store_counts = filtered_data['Zip Code'].value_counts()
print("Store counts by zip code:")
print(store_counts)

# Extract latitude and longitude from 'Georeference' column
filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)

# Create a map centered around the NYC area
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Add markers for each store
for _, row in filtered_data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['DBA Name']
    ).add_to(nyc_map)

# Display the map
nyc_map.save("nyc_retail_map.html")


Store counts by zip code:
Zip Code
11211    150
11206    124
11101     96
10001     46
11249     43
10012     38
10036     37
Name: count, dtype: int64


C:\Users\zheng\AppData\Local\Temp\ipykernel_1236\2802522985.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)
C:\Users\zheng\AppData\Local\Temp\ipykernel_1236\2802522985.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)


In [19]:
# Group by Zip Code and DBA Name, and get counts
grouped_data = filtered_data.groupby(['Zip Code', 'Entity Name']).size().reset_index(name='Count')

# Save the grouped data to CSV
grouped_data.to_csv("store_counts_by_zipcode.csv", index=False)

print("Data saved to store_counts_by_zipcode.csv")

Data saved to store_counts_by_zipcode.csv


In [18]:
# Convert Zip Code column to string and strip any whitespace
data['Zip Code'] = data['Zip Code'].astype(str).str.strip()

# Check unique values in Zip Code for debugging
print("Unique zip codes in the data:")
print(data['Zip Code'].unique())

# List of target zip codes
target_zipcodes = ['10036', '10001', '10012', '11101', '11206', '11211', '11249']

# Filter for stores in the target zip codes
filtered_data = data[data['Zip Code'].isin(target_zipcodes)]
print("\nFiltered Data:")
print(filtered_data.head())  # Debugging step to check if data is filtered correctly

# Group by Zip Code and count unique DBA Names (stores) in each zip code
store_counts = filtered_data.groupby('Zip Code')['DBA Name'].nunique().reindex(target_zipcodes, fill_value=0)
print("\nStore Counts by Zip Code:")
print(store_counts)  # Debugging step to check the store counts

# Plotting
plt.figure(figsize=(10, 6))
store_counts.plot(kind='bar', color='skyblue')
plt.title("Number of Stores by Zip Code")
plt.xlabel("Zip Code")
plt.ylabel("Number of Stores")
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show(block=True)

Unique zip codes in the data:
['11729' '13827' '10598' ... '12085' '12186' '12067']

Filtered Data:
       County  License Number Operation Type Establishment Type  \
16   NEW YORK          762969          Store                  A   
17   NEW YORK          762966          Store                  A   
18   NEW YORK          762964          Store                  A   
86     QUEENS          762293          Store                 AC   
186  NEW YORK          761783          Store                 AC   

                      Entity Name                DBA Name Street Number  \
16   HUDSON GROUP (HG) RETAIL LLC  HUDSON NEWS STORE #210           NaN   
17   HUDSON GROUP (HG) RETAIL LLC  HUDSON NEWS STORE #211           NaN   
18   HUDSON GROUP (HG) RETAIL LLC  HUDSON NEWS STORE #212           NaN   
86    EAST ASIAN FOOD CORPORATION         EAST ASIAN FOOD          2802   
186     PARTNERS IN CHOCOLATE LLC   PARTNERS IN CHOCOLATE           456   

             Street Name Address Line 2 Addres

KeyboardInterrupt: 

In [21]:
# List of target zip codes
target_zipcodes = ['10036', '10001', '10012', '11101', '11206', '11211', '11249']

# Keywords to search for in DBA Name
keywords = [
    "Deli", "Grocer", "Bakery", "Bistro", "Café", "Sweets", "Grill", 
    "Mart", "Shop", "Market", "Corner Store", "Quick Stop", 
    "Mini", "Little", "Cozy", "Old", "New", "Tiny", "Small", "Family"
]

# Combine keywords into a regex pattern
pattern = '|'.join(keywords)

# Convert Zip Code column to string for filtering and strip any whitespace
data['Zip Code'] = data['Zip Code'].astype(str).str.strip()

# Apply filters: target zip codes and DBA Name keywords
filtered_data = data[
    data['Zip Code'].isin(target_zipcodes) & 
    data['DBA Name'].str.contains(pattern, case=False, na=False)
]

# Extract latitude and longitude from 'Georeference' column
filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)

# Create a map centered around the NYC area
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Add markers for each store
for _, row in filtered_data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['DBA Name']
    ).add_to(nyc_map)

# Save the map as an HTML file
nyc_map.save("filtered_retail_map.html")
print("Map saved as filtered_retail_map.html")

C:\Users\zheng\AppData\Local\Temp\ipykernel_1236\2378037374.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)
C:\Users\zheng\AppData\Local\Temp\ipykernel_1236\2378037374.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data[['Longitude', 'Latitude']] = filtered_data['Georeference'].str.extract(r'POINT \(([^ ]+) ([^ ]+)\)').astype(float)


Map saved as filtered_retail_map.html


In [23]:
# List of target zip codes
target_zipcodes = ['10036', '10001', '10012', '11101', '11206', '11211', '11249']

# Keywords to search for in DBA Name
keywords = [
    "Deli", "Grocer", "Bakery", "Bistro", "Café", "Sweets", "Grill", 
    "Mart", "Shop", "Market", "Corner Store", "Quick Stop", 
    "Mini", "Little", "Cozy", "Old", "New", "Tiny", "Small", "Family"
]

# Combine keywords into a regex pattern
pattern = '|'.join(keywords)

# Convert Zip Code column to string for filtering and strip any whitespace
data['Zip Code'] = data['Zip Code'].astype(str).str.strip()

# Initialize a dictionary to store the percentage results
zip_code_percentages = {}

# Loop through each target zip code
for zip_code in target_zipcodes:
    # Filter data for the current zip code
    zip_data = data[data['Zip Code'] == zip_code]
    total_stores_in_zip = len(zip_data)
    
    # Filter stores that also match the keywords within this zip code
    matching_stores_in_zip = zip_data[zip_data['DBA Name'].str.contains(pattern, case=False, na=False)]
    filtered_count = len(matching_stores_in_zip)
    
    # Calculate percentage for this zip code
    percentage = (filtered_count / total_stores_in_zip * 100) if total_stores_in_zip > 0 else 0
    zip_code_percentages[zip_code] = percentage

# Display the percentages for each zip code
for zip_code, pct in zip_code_percentages.items():
    print(f"Zip Code {zip_code}: {pct:.2f}% of stores match the criteria")

Zip Code 10036: 45.95% of stores match the criteria
Zip Code 10001: 45.65% of stores match the criteria
Zip Code 10012: 34.21% of stores match the criteria
Zip Code 11101: 41.67% of stores match the criteria
Zip Code 11206: 62.10% of stores match the criteria
Zip Code 11211: 60.00% of stores match the criteria
Zip Code 11249: 51.16% of stores match the criteria
